In [4]:
using FFTW

function ricker(input::Array; dt=1.0, f=0.2)  
    # Fourier transform
    fourier = rfft(input)
    nt = length(input)
    freq = rfftfreq(nt, 1/dt)

    # Ricker wavelet
    freq = freq/f
    freq = freq .* freq
    @. fourier *= freq * exp(1 - freq)
    return irfft(fourier, nt)
end

   input = [0, 0, 0, 1, 0, 0, 0]  
    dt = 0.1                        
    f = 10.0                       

   
ricker(input; dt, f) 


7-element Vector{Float64}:
 -0.003977750622246395
  0.01791785195573209
 -0.11027749899418056
  0.19267479532138973
 -0.11027749899418057
  0.017917851955732096
 -0.003977750622246393